For generating abstractive text summarization, Transformer model is used. Its part of Deep learning methodolgy. Using transfer learning, t5model within Transformers is imported and fine tuned for training, test data. To implement, oad dataset is created using Pytorch framework, importing Dataset class and overriding len and getitem methods. Within getitem, the dataset is accessed and t5tokenizer is applied to convert to tokenids along with attention mask. This is performed on both source text and target summary.

In [ ]:
!pip install pytorch-lightning --quiet
!pip install nlp
!pip install --upgrade pyarrow
!pip install transformers
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from nlp import load_metric

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

Requirement already satisfied: nlp in /usr/local/lib/python3.6/dist-packages (0.4.0)
Requirement already satisfied: dataclasses; python_version < "3.7" in /usr/local/lib/python3.6/dist-packages (from nlp) (0.8)
Requirement already satisfied: xxhash in /usr/local/lib/python3.6/dist-packages (from nlp) (2.0.0)
Requirement already satisfied: pandas in /usr/local/lib/python3.6/dist-packages (from nlp) (1.1.5)
Requirement already satisfied: requests>=2.19.0 in /usr/local/lib/python3.6/dist-packages (from nlp) (2.23.0)
Requirement already satisfied: numpy in /usr/local/lib/python3.6/dist-packages (from nlp) (1.19.5)
Requirement already satisfied: filelock in /usr/local/lib/python3.6/dist-packages (from nlp) (3.0.12)
Requirement already satisfied: dill in /usr/local/lib/python3.6/dist-packages (from nlp) (0.3.3)
Requirement already satisfied: pyarrow>=0.16.0 in /usr/local/lib/python3.6/dist-packages (from nlp) (3.0.0)
Requirement already satisfied: tqdm>=4.27 in /usr/local/lib/python3.6/dist-packages (from nlp) (4.41.1)
Requirement already satisfied: pytz>=2017.2 in /usr/local/lib/python3.6/dist-packages (from pandas->nlp) (2018.9)
Requirement already satisfied: python-dateutil>=2.7.3 in /usr/local/lib/python3.6/dist-packages (from pandas->nlp) (2.8.1)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from requests>=2.19.0->nlp) (1.24.3)
Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.6/dist-packages (from requests>=2.19.0->nlp) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests>=2.19.0->nlp) (2020.12.5)
Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.6/dist-packages (from requests>=2.19.0->nlp) (3.0.4)
Requirement already satisfied: six>=1.5 in /usr/local/lib/python3.6/dist-packages (from python-dateutil>=2.7.3->pandas->nlp) (1.15.0)
Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (3.0.0)
Requirement already satisfied, skipping upgrade: numpy>=1.16.6 in /usr/local/lib/python3.6/dist-packages (from pyarrow) (1.19.5)
Requirement already satisfied: transformers in /usr/local/lib/python3.6/dist-packages (4.3.2)
Requirement already satisfied: regex!=2019.12.17 in /usr/local/lib/python3.6/dist-packages (from transformers) (2019.12.20)
Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.6/dist-packages (from transformers) (1.19.5)
Requirement already satisfied: tqdm>=4.27 in /usr/local/lib/python3.6/dist-packages (from transformers) (4.41.1)
Requirement already satisfied: importlib-metadata; python_version < "3.8" in /usr/local/lib/python3.6/dist-packages (from transformers) (3.4.0)
Requirement already satisfied: tokenizers<0.11,>=0.10.1 in /usr/local/lib/python3.6/dist-packages (from transformers) (0.10.1)
Requirement already satisfied: requests in /usr/local/lib/python3.6/dist-packages (from transformers) (2.23.0)
Requirement already satisfied: packaging in /usr/local/lib/python3.6/dist-packages (from transformers) (20.9)
Requirement already satisfied: sacremoses in /usr/local/lib/python3.6/dist-packages (from transformers) (0.0.43)
Requirement already satisfied: dataclasses; python_version < "3.7" in /usr/local/lib/python3.6/dist-packages (from transformers) (0.8)
Requirement already satisfied: filelock in /usr/local/lib/python3.6/dist-packages (from transformers) (3.0.12)
Requirement already satisfied: typing-extensions>=3.6.4; python_version < "3.8" in /usr/local/lib/python3.6/dist-packages (from importlib-metadata; python_version < "3.8"->transformers) (3.7.4.3)
Requirement already satisfied: zipp>=0.5 in /usr/local/lib/python3.6/dist-packages (from importlib-metadata; python_version < "3.8"->transformers) (3.4.0)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from requests->transformers) (1.24.3)
Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.6/dist-packages (from requests->transformers) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.6/dist-packages (from requests->transformers) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests->transformers) (2020.12.5)
Requirement already satisfied: pyparsing>=2.0.2 in /usr/local/lib/python3.6/dist-packages (from packaging->transformers) (2.4.7)
Requirement already satisfied: click in /usr/local/lib/python3.6/dist-packages (from sacremoses->transformers) (7.1.2)
Requirement already satisfied: joblib in /usr/local/lib/python3.6/dist-packages (from sacremoses->transformers) (1.0.0)
Requirement already satisfied: six in /usr/local/lib/python3.6/dist-packages (from sacremoses->transformers) (1.15.0)

In [ ]:
!pip install wandb
import wandb
YOUR_API_KEY = 'b9ccbb5790e3b4aaac2227be50d35544df4dae18'
os.environ["WANDB_API_KEY"] = YOUR_API_KEY


Load dataset is created using Pytorch framework, importing Dataset class and overriding len and getitem methods. Within getitem, the dataset is accessed and t5tokenizer is applied to convert to tokenids along with attention mask. This is performed on both source text and target summary.

In [ ]:
class Load_dataset(Dataset):
    def __init__(self, dataframe, tokenizer, num_samples, input_length, output_length, print_text=False):         
        self.dataset = dataframe
        #self.dataset =  news_df2
        if num_samples:
            self.dataset = self.dataset.iloc[:num_samples]
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
        self.print_text = print_text
  
    def __len__(self):
        return self.dataset.shape[0]
    
    
    def clean_text(self, text):
        text = text.replace('\n','')
        text = text.replace('``', '')
        text = text.replace('"', '')
        
        return text
    
    
    def convert_to_features(self, example_batch):
        # Tokenize contexts and questions (as pairs of inputs)
        
        if self.print_text:
            print("Input Text: ", self.clean_text(example_batch['clean_text']))
#         input_ = self.clean_text(example_batch['text']) + " </s>"
#         target_ = self.clean_text(example_batch['headline']) + " </s>"
        
        input_ = self.clean_text(example_batch['clean_text'])
        if self.print_text:
            print("Input: ", input_)
            print("Input length: ", self.input_length)
        target_ = self.clean_text(example_batch['clean_sum'])
        
        source = self.tokenizer.batch_encode_plus([input_], max_length=self.input_length, padding='max_length', truncation=True, return_tensors="pt")
        
        targets = self.tokenizer.batch_encode_plus([target_], max_length=self.output_length, padding='max_length', truncation=True, return_tensors="pt")
    
       
        return source, targets
  
    def __getitem__(self, index):
        source, targets = self.convert_to_features(self.dataset.iloc[index])
        
        source_ids = source["input_ids"].squeeze()
        target_ids = targets["input_ids"].squeeze()
        #target_ids[target_ids[:,:] == self.tokenizer.pad_token_id] =-100

        src_mask    = source["attention_mask"].squeeze()
        target_mask = targets["attention_mask"].squeeze()
        

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

Install sentencepice and T5 tokenizer and Conditionforgeneration models from transfromer huggingface library. Here t5-base is used for better performance

In [ ]:
!pip install sentencepiece
!pip install tf-sentencepiece
tokenizer = T5Tokenizer.from_pretrained('t5-base')
#dataset = Load_dataset(tokenizer, None, 512, 150, True)

Requirement already satisfied: sentencepiece in /usr/local/lib/python3.6/dist-packages (0.1.95)
Requirement already satisfied: tf-sentencepiece in /usr/local/lib/python3.6/dist-packages (0.1.90)
HBox(children=(FloatProgress(value=0.0, description='Downloading', max=791656.0, style=ProgressStyle(descripti…

Train and validate methods are defied which accept epoch, tokenizer along with model, loader and optimizer. During our model, T5forConditionalgenerator is used and trained by passing input_ids of source text, source attention mask along with decoder input ids and labels. Adam Optimizer is performed with loss is calculated. During test step, model.generate is used to output generated_ids by using tuned param of beam search as 2, repetition_penalty=2.5, length_penalty=1.0. Then generated ids are decoded into predicted summary.

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

Main method invokes training configuraion with batch size, epoch size and learnin rate. Train and test mmodel methods are invoked by splitting datafrae into train/test sizes and invoking dataloader dataset to pass relevenat datasets accodingly.

In [ ]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="transformers_tutorials_summarization")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 4    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 4    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 3e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150 
    config.adam_epsilon=1e-8

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    #df = pd.read_csv('./data/news_summary.csv',encoding='latin-1')
    #df = df[['text','ctext']]
    df= news_df2
    df.clean_text = 'summarize: ' + df.clean_text
    print(news_df2.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = Load_dataset(train_dataset, tokenizer,None, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = Load_dataset(val_dataset, tokenizer, None, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE, eps=config.adam_epsilon)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    #C:\Users\sa182c\Desktop\personal\python\projects\dailymail\saved_model
    #import os
    torch.save(model, "/content/sample_data/t5")
    #saved_model_dir = f"/content/{MODEL}"

    #saved_model_path = os.path.join(saved_model_dir, max(os.listdir(saved_model_dir)))
    

if __name__ == '__main__':
    main()

Tracking run with wandb version 0.10.18
Syncing run sleek-salad-33 to Weights & Biases (Documentation).
Project page: https://wandb.ai/swathikakula/transformers_tutorials_summarization
Run page: https://wandb.ai/swathikakula/transformers_tutorials_summarization/runs/3cim5ij7
Run data is saved locally in /content/wandb/run-20210214_155012-3cim5ij7

                                           clean_sum                                         clean_text
0  the 45yearold has hit out against pressure to ...  summarize: despite the huge pressure upon wome...
1  scarlets hooker ken owens has won 26 caps with...  summarize: wales have called scarlets hooker k...
2  joanne stronach worked in cafe at yorkshire de...  summarize: a motherofthree was sacked from her...
3  information has been redacted from a report in...  summarize: by daniel mills for daily mail aust...
4  bella rodrigueztorres was diagnosed with stage...  summarize: by helen pow published 0916 est 29 ...
FULL Dataset: (152, 2)
TRAIN Dataset: (122, 2)
TEST Dataset: (30, 2)
Initiating Fine-Tuning for the model on our dataset
Epoch: 0, Loss:  6.280526638031006